## Q1) Read and explore the given dataset.  ( Rename column/add headers, plot histograms, find data characteristics)

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter
import sklearn.model_selection
from sklearn.model_selection import cross_validate
from sklearn.metrics import accuracy_score
from surprise import KNNBasic, SVD, SVDpp, SlopeOne, NMF, NormalPredictor, KNNBaseline, \
KNNWithMeans, KNNWithZScore, BaselineOnly, CoClustering,evaluate, Reader, dataset, accuracy
from __future__ import (absolute_import, division, print_function,
                        unicode_literals)
from surprise.model_selection import train_test_split

In [6]:
ratings_df = pd.read_csv("mine/ratings_Electronics.csv")

In [9]:
ratings_df.sample(5)

,AKM1MP6P0OYPR,0132793040,5.0,1365811200
3296205,ANIZWD1EXXGAI,B003ELYQGG,4.0,1351555200
1620775,AG35JCCQWDRCS,B0012WXFO8,5.0,1375920000
739698,A32E87FU6973FF,B000B9O83A,3.0,1205539200
952849,AS5U0DWZLF00Y,B000FUOSF8,1.0,1405468800
314873,A31R3ZYX2OT7FI,B0000ADY4L,4.0,1069545600


In [8]:
ratings_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7824481 entries, 0 to 7824480
Data columns (total 4 columns):
AKM1MP6P0OYPR    object
0132793040       object
5.0              float64
1365811200       int64
dtypes: float64(1), int64(1), object(2)
memory usage: 238.8+ MB


In [10]:
ratings_df.columns = ['userID', 'itemID', 'rating', 'timestamp']

In [11]:
ratings_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7824481 entries, 0 to 7824480
Data columns (total 4 columns):
userID       object
itemID       object
rating       float64
timestamp    int64
dtypes: float64(1), int64(1), object(2)
memory usage: 238.8+ MB


In [12]:
ratings_df.head()

,userID,itemID,rating,timestamp
0,A2CX7LUOHB2NDG,0321732944,5.0,1341100800
1,A2NWSAGRHCP8N5,0439886341,1.0,1367193600
2,A2WNBOD3WNDNKT,0439886341,3.0,1374451200
3,A1GI0U4ZRJA8WN,0439886341,1.0,1334707200
4,A1QGNMC6O1VW39,0511189877,5.0,1397433600


## Q2) Take a subset of the dataset to make it less sparse/ denser. ( For example, keep the users only who has given 50 or more number of ratings )

In [13]:
user_filter = ratings_df.groupby("userID").agg({"rating":"count"})>=50

In [14]:
user_filter.head()

,rating
userID,
A00000262KYZUE4J55XGL,False
A000063614T1OE0BUSKUT,False
A00009182QVLSWIGHLS1B,False
A00009661LC9LQPGKJ24G,False
A00010809P09NUU6ZP6H,False


In [15]:
print("Total Number of users are {}".format(user_filter.shape[0]))

Total Number of users are 4201696


In [16]:
user_filter_50 = user_filter[user_filter["rating"] == True]

In [17]:
user_filter_50.shape

(1540, 1)

In [18]:
user_filter_50 = user_filter_50.reset_index()

In [19]:
ratings_df_filtered = ratings_df[ratings_df["userID"].isin(user_filter_50["userID"])]

In [20]:
ratings_df_filtered.shape

(125871, 4)

In [21]:
ratings_df_filtered.head()

,userID,itemID,rating,timestamp
93,A3BY5KCNQZXV5U,0594451647,5.0,1390176000
117,AT09WGFUM934H,0594481813,3.0,1377907200
176,A32HSNCNPRUMTR,0970407998,1.0,1319673600
177,A17HMM1M7T9PJ1,0970407998,4.0,1281744000
491,A3CLWR1UUZT6TG,0972683275,5.0,1373587200


In [22]:
ratings_df_filtered.drop("timestamp", axis=1, inplace=True)

C:\Users\kumar\Anaconda3\lib\site-packages\pandas\core\frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [23]:
ratings_df_filtered.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 125871 entries, 93 to 7824443
Data columns (total 3 columns):
userID    125871 non-null object
itemID    125871 non-null object
rating    125871 non-null float64
dtypes: float64(1), object(2)
memory usage: 3.8+ MB


In [24]:
product_frequency = Counter(ratings_df_filtered["itemID"])

In [25]:
product_frequency = sorted(product_frequency.items(), key=lambda x: x[1],reverse=True)

In [26]:
product_frequency

[('B0088CJT4U', 206),
 ('B003ES5ZUU', 184),
 ('B000N99BBC', 167),
 ('B007WTAJTO', 164),
 ('B00829TIEK', 149),
 ('B00829THK0', 137),
 ('B008DWCRQW', 137),
 ('B002R5AM7C', 128),
 ('B004CLYEDC', 118),
 ('B002SZEOLG', 113),
 ('B00834SJNA', 110),
 ('B004T9RR6I', 109),
 ('B002V88HFE', 106),
 ('B0034CL2ZI', 106),
 ('B00834SJSK', 101),
 ('B002WE6D44', 100),
 ('B00829TIA4', 99),
 ('B004CLYEFK', 97),
 ('B009SYZ8OC', 95),
 ('B0079UAT0A', 92),
 ('B0019EHU8G', 90),
 ('B005CT56F8', 87),
 ('B000QUUFRW', 84),
 ('B00DR0PDNE', 84),
 ('B00HFRWWAM', 82),
 ('B005FYNSPK', 81),
 ('B004CLYEE6', 80),
 ('B0041Q38NU', 79),
 ('B001TH7GUU', 78),
 ('B00ARB5FLQ', 78),
 ('B002SQK2F2', 77),
 ('B005HMKKH4', 77),
 ('B0015DYMVO', 76),
 ('B004CLYEH8', 76),
 ('B00AQRUW4Q', 76),
 ('B003M0NURK', 74),
 ('B005CLPP84', 74),
 ('B00004ZCJE', 72),
 ('B000LRMS66', 71),
 ('B000VX6XL6', 70),
 ('B001E1Y5O6', 70),
 ('B001XURP7W', 69),
 ('B00BOHNYTW', 69),
 ('B0002L5R78', 67),
 ('B001CHHVCU', 67),
 ('B00884WH74', 66),
 ('B000JMJWV2', 64

In [27]:
x = []
y = []
for item in product_frequency:
    x.append(item[0])
    y.append(item[1])

## Q3) Split the data randomly into train and test dataset. ( For example, split it in 70/30 ratio)

In [28]:
train_data, test_data = sklearn.model_selection.train_test_split(ratings_df_filtered, test_size = 0.30)

## Q4) Build Popularity Recommender model.

In [29]:
agg_product_ratings = ratings_df_filtered.groupby("itemID").agg({"rating":"sum"})
agg_product_ratings = agg_product_ratings.reset_index()
agg_product_ratings = agg_product_ratings.sort_values(by="rating",ascending=False)

In [30]:
def return_popular_products(userID, n=10):
    user_rated_items = ratings_df_filtered[ratings_df_filtered["userID"]==userID]["itemID"]
    user_recommendations = agg_product_ratings[~agg_product_ratings["itemID"].isin(user_rated_items)]
    return user_recommendations.head(n)

In [31]:
userID = "A17HMM1M7T9PJ1"
n = 20
print("TOP {} popular products based on their ratings are for user {}:\n".format(n, userID))
print(return_popular_products(userID, n))

TOP 20 popular products based on their ratings are for user A17HMM1M7T9PJ1:

           itemID  rating
24827  B003ES5ZUU   895.0
39003  B0088CJT4U   869.0
11078  B000N99BBC   797.0
38250  B007WTAJTO   771.0
38615  B00829TIEK   661.0
38611  B00829THK0   605.0
39338  B008DWCRQW   561.0
28761  B004CLYEDC   551.0
21982  B002R5AM7C   528.0
22460  B002V88HFE   498.0
38655  B00834SJNA   494.0
23378  B0034CL2ZI   491.0
22210  B002SZEOLG   480.0
22602  B002WE6D44   477.0
38656  B00834SJSK   469.0
38614  B00829TIA4   452.0
28763  B004CLYEFK   448.0
15538  B0019EHU8G   437.0
41618  B009SYZ8OC   425.0
31001  B004T9RR6I   422.0


## 5) Build Collaborative Filtering model.

In [172]:
reader = Reader(line_format='user item rating', rating_scale=(1, 5))

In [173]:
class MyDataset(dataset.DatasetAutoFolds):

    def __init__(self, df, reader):

        self.raw_ratings = [(uid, iid, r, None) for (uid, iid, r) in
                            zip(df['userID'], df['itemID'], df['rating'])]
        self.reader=reader

data = MyDataset(ratings_df_filtered, reader)

In [82]:
benchmark = []
data.split(n_folds=3)
# Iterate over all algorithms
for algorithm in [SVD(), SVDpp(), SlopeOne(), NMF(), NormalPredictor(), KNNBaseline(), KNNBasic(), KNNWithMeans(), KNNWithZScore(), BaselineOnly(), CoClustering()]:
    # Perform cross validation
    results = evaluate(algorithm, data, measures=['RMSE', 'MAE'])
    
    # Get results & append algorithm name
    tmp = pd.DataFrame.from_dict(results).mean(axis=0)
    tmp = tmp.append(pd.Series([str(algorithm).split(' ')[0].split('.')[-1]], index=['Algorithm']))
    benchmark.append(tmp)
    
pd.DataFrame(benchmark).set_index('Algorithm').sort_values('rmse')

/Users/mac/anaconda3/lib/python3.7/site-packages/surprise/evaluate.py:66: UserWarning: The evaluate() method is deprecated. Please use model_selection.cross_validate() instead.
  'model_selection.cross_validate() instead.', UserWarning)
/Users/mac/anaconda3/lib/python3.7/site-packages/surprise/dataset.py:193: UserWarning: Using data.split() or using load_from_folds() without using a CV iterator is now deprecated. 
  UserWarning)


Evaluating RMSE, MAE of algorithm SVD.

------------
Fold 1
RMSE: 0.9867
MAE:  0.7316
------------
Fold 2
RMSE: 0.9862
MAE:  0.7291
------------
Fold 3
RMSE: 0.9841
MAE:  0.7310
------------
------------
Mean RMSE: 0.9857
Mean MAE : 0.7305
------------
------------
Evaluating RMSE, MAE of algorithm SVDpp.



/Users/mac/anaconda3/lib/python3.7/site-packages/surprise/evaluate.py:66: UserWarning: The evaluate() method is deprecated. Please use model_selection.cross_validate() instead.
  'model_selection.cross_validate() instead.', UserWarning)
/Users/mac/anaconda3/lib/python3.7/site-packages/surprise/dataset.py:193: UserWarning: Using data.split() or using load_from_folds() without using a CV iterator is now deprecated. 
  UserWarning)


------------
Fold 1
RMSE: 0.9861
MAE:  0.7278
------------
Fold 2
RMSE: 0.9869
MAE:  0.7253
------------
Fold 3
RMSE: 0.9858
MAE:  0.7279
------------
------------
Mean RMSE: 0.9863
Mean MAE : 0.7270
------------
------------
Evaluating RMSE, MAE of algorithm SlopeOne.



/Users/mac/anaconda3/lib/python3.7/site-packages/surprise/evaluate.py:66: UserWarning: The evaluate() method is deprecated. Please use model_selection.cross_validate() instead.
  'model_selection.cross_validate() instead.', UserWarning)
/Users/mac/anaconda3/lib/python3.7/site-packages/surprise/dataset.py:193: UserWarning: Using data.split() or using load_from_folds() without using a CV iterator is now deprecated. 
  UserWarning)


...

/Users/mac/anaconda3/lib/python3.7/site-packages/surprise/evaluate.py:66: UserWarning: The evaluate() method is deprecated. Please use model_selection.cross_validate() instead.
  'model_selection.cross_validate() instead.', UserWarning)
/Users/mac/anaconda3/lib/python3.7/site-packages/surprise/dataset.py:193: UserWarning: Using data.split() or using load_from_folds() without using a CV iterator is now deprecated. 
  UserWarning)


------------
Fold 1
Estimating biases using als...
RMSE: 0.9823
MAE:  0.7368
------------
Fold 2
Estimating biases using als...
RMSE: 0.9833
MAE:  0.7347
------------
Fold 3
Estimating biases using als...
RMSE: 0.9803
MAE:  0.7356
------------
------------
Mean RMSE: 0.9820
Mean MAE : 0.7357
------------
------------
Evaluating RMSE, MAE of algorithm CoClustering.

------------
Fold 1


/Users/mac/anaconda3/lib/python3.7/site-packages/surprise/evaluate.py:66: UserWarning: The evaluate() method is deprecated. Please use model_selection.cross_validate() instead.
  'model_selection.cross_validate() instead.', UserWarning)
/Users/mac/anaconda3/lib/python3.7/site-packages/surprise/dataset.py:193: UserWarning: Using data.split() or using load_from_folds() without using a CV iterator is now deprecated. 
  UserWarning)


RMSE: 1.0780
MAE:  0.7683
------------
Fold 2
RMSE: 1.0897
MAE:  0.7727
------------
Fold 3
RMSE: 1.0825
MAE:  0.7699
------------
------------
Mean RMSE: 1.0834
Mean MAE : 0.7703
------------
------------


,rmse,mae
Algorithm,,
BaselineOnly,0.981997,0.735664
SVD,0.985663,0.730543
SVDpp,0.986278,0.727021
KNNBaseline,1.044604,0.751276
KNNWithMeans,1.065912,0.771968
KNNWithZScore,1.069251,0.768743
CoClustering,1.083413,0.770297
SlopeOne,1.096082,0.809033
KNNBasic,1.113206,0.804236


In [83]:
pd.DataFrame(benchmark).set_index('Algorithm').sort_values('rmse')

,rmse,mae
Algorithm,,
BaselineOnly,0.981997,0.735664
SVD,0.985663,0.730543
SVDpp,0.986278,0.727021
KNNBaseline,1.044604,0.751276
KNNWithMeans,1.065912,0.771968
KNNWithZScore,1.069251,0.768743
CoClustering,1.083413,0.770297
SlopeOne,1.096082,0.809033
KNNBasic,1.113206,0.804236


## BaselineOnly gave best rmse among all, so Lets use BaselineOnly for training.

## 6) Evaluate both the models. ( Once the model is trained on the training data, it can be used to compute the error (RMSE) on predictions made on the test data.)

In [105]:

print('Using ALS(Alternating least squares)')
bsl_options = {'method': 'als',
               'n_epochs': 5,
               'reg_u': 12,
               'reg_i': 5
               }
algo = BaselineOnly(bsl_options=bsl_options)
evaluate(algo, data, measures=['RMSE'], verbose=False)

Using ALS(Alternating least squares)
Estimating biases using als...


/Users/mac/anaconda3/lib/python3.7/site-packages/surprise/evaluate.py:66: UserWarning: The evaluate() method is deprecated. Please use model_selection.cross_validate() instead.
  'model_selection.cross_validate() instead.', UserWarning)
/Users/mac/anaconda3/lib/python3.7/site-packages/surprise/dataset.py:193: UserWarning: Using data.split() or using load_from_folds() without using a CV iterator is now deprecated. 
  UserWarning)


Estimating biases using als...
Estimating biases using als...


CaseInsensitiveDefaultDict(list,
                           {'rmse': [0.980047607740335,
                             0.9811552196886554,
                             0.9787144739202497]})

In [130]:
trainset, testset = train_test_split(data, test_size=.25)
algo = BaselineOnly(bsl_options=bsl_options)

In [135]:
predictions = algo.fit(trainset).test(testset)
accuracy.rmse(predictions)

Estimating biases using als...
RMSE: 0.9793


0.9793148111684349

## 7) Get top - K ( K = 5) recommendations. Since our goal is to recommend new products to each user based on his/her habits, we will recommend 5 new products.

In [174]:
userID = "A1QGNMC6O1VW39"
n = 5
print("TOP {} popular products based on their ratings are for user {}:\n".format(n, userID))
print(return_popular_products(userID, n))

TOP 5 popular products based on their ratings are for user A1QGNMC6O1VW39:

           itemID  rating
24827  B003ES5ZUU   895.0
39003  B0088CJT4U   869.0
11078  B000N99BBC   797.0
38250  B007WTAJTO   771.0
38615  B00829TIEK   661.0


In [136]:
def get_Iu(uid):
    """ return the number of items rated by given user
    args: 
      uid: the id of the user
    returns: 
      the number of items rated by the user
    """
    try:
        return len(trainset.ur[trainset.to_inner_uid(uid)])
    except ValueError: # user was not part of the trainset
        return 0
    
def get_Ui(iid):
    """ return number of users that have rated given item
    args:
      iid: the raw id of the item
    returns:
      the number of users that have rated the item.
    """
    try: 
        return len(trainset.ir[trainset.to_inner_iid(iid)])
    except ValueError:
        return 0
    
df = pd.DataFrame(predictions, columns=['uid', 'iid', 'rui', 'est', 'details'])
df['Iu'] = df.uid.apply(get_Iu)
df['Ui'] = df.iid.apply(get_Ui)
df['err'] = abs(df.est - df.rui)
best_predictions = df.sort_values(by='err')[:10]
worst_predictions = df.sort_values(by='err')[-10:]

In [137]:
best_predictions

,uid,iid,rui,est,details,Iu,Ui,err
24266,AGLT3L7ALL4O4,B000L47AHG,5.0,5.0,{'was_impossible': False},44,20,0.0
7335,A3963R7EPE3A7E,B002WE4HE2,5.0,5.0,{'was_impossible': False},50,44,0.0
20286,ABDR6IJ93HFIO,B00009EHJV,5.0,5.0,{'was_impossible': False},148,9,0.0
361,A3S3R88HA0HZG3,B003ES5ZUU,5.0,5.0,{'was_impossible': False},110,139,0.0
14017,A2NOW4U7W3F7RI,B00HWMPSF6,5.0,5.0,{'was_impossible': False},213,8,0.0
16882,A2XRMQA6PJ5ZJ8,B009USAJCC,5.0,5.0,{'was_impossible': False},203,14,0.0
22665,AYB4ELCS5AM8P,B004CLYEFK,5.0,5.0,{'was_impossible': False},126,78,0.0
7200,A3D0UM4ZD2CMAW,B005J7YA4G,5.0,5.0,{'was_impossible': False},87,10,0.0
3675,A1PV5T5S4D5NPI,B00316263Y,5.0,5.0,{'was_impossible': False},69,27,0.0
16933,A25FL6VLD7S23S,B0012IJYZQ,5.0,5.0,{'was_impossible': False},83,6,0.0


## 8) Summarise your insights.

#### 1. The above are the best predictions, and they are not lucky guesses. Because Ui is anywhere between 6 to 138, they are not really small, meaning that significant number of users have rated the target book.

#### 2. Colloborative filtering implemented above with the help of surprise library. It can be implemented manually and by manually calculating covariance matrix 